#### This module loads pickled url files and unpacks them into one list item then visits the links and scrapes the data

In [76]:
import pickle as pk
import os
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as conditions
from selenium.webdriver.common.by import By



In [77]:
files = os.listdir('links/')

In [78]:
urls = []
for file in files:
    _ = open(('links/'+file),'rb')
    urls.append(pk.load(_))

urls = [url for sublist in urls for url in sublist]
# len(urls)

In [79]:
options = Options()
options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36')
options.add_argument("--headless")
options.add_argument("--disable-blink-features=AutomationControlled")
driver = webdriver.Chrome(options=options)

In [80]:
def parser(dirty):
    
    text = BeautifulSoup(dirty,'html.parser').get_text()
    cleaned = text.replace("\n", " ").replace("\xa0", " ")
    return cleaned
    

In [81]:
try:
    jobs = {}
    xpath = {'company':'//h2[@class="pb-1 text-sm font-normal"]',
             'title':'//*[@id="tab1"]/div/article/div[2]/div[2]/h1',
             'description':"//*[@id='tab1']/div/article/div[5]/div",
             'location':'//*[@id="tab1"]/div/article/div[2]/div[2]/div[1]/*[1]',
             'nature':'//*[@id="tab1"]/div/article/div[2]/div[2]/div[1]/*[2]',
             'salary':'//*[@id="tab1"]/div/article/div[2]/div[2]/div[2]/span[1]/span',
             'posted':'//*[@id="tab1"]/div/article/div[3]/div[2]'
             }
        
        
    driver.get(urls[37])
    print(f"Going to {urls[37]}")
    WebDriverWait(driver, 10).until(conditions.presence_of_element_located((By.XPATH, xpath['description'])),)
    
    #  get job details and clean innerHTML from posted and description 
    company = driver.find_element_by_xpath(xpath["company"])
    title = driver.find_element_by_xpath(xpath["title"])
    location = driver.find_element_by_xpath(xpath['location'])
    nature = driver.find_element_by_xpath(xpath['nature'])
    salary = driver.find_element_by_xpath(xpath["salary"])
    posted = parser(driver.find_element_by_xpath(xpath["posted"]).get_attribute("innerHTML")) 
    description = parser(driver.find_element_by_xpath(xpath['description']).get_attribute("innerHTML"))
     
    jobs.update({'company':company.text,
                 'title':title.text,
                 'location':location.text,
                 'nature':nature.text,
                 'salary': (salary.text if len(salary.text) > 6 else "Unspecified"),
                 'description':description,
                 'posted':posted
                 })
    
except NoSuchElementException:
    print("Hii stuff haiko buda")
    
driver.quit()
        

Going to https://www.brightermonday.co.ke/listings/senior-secops-engineer-np5gpw


In [82]:
jobs

{'company': 'Tala',
 'title': 'Senior SecOps Engineer',
 'location': 'Nairobi',
 'nature': 'Full Time',
 'salary': 'Unspecified',
 'description': "  About Tala Tala is a global technology company building the worldâs most accessible financial services. With more than $350 million raised from visionary investors, we are serving millions of customers around the world who have been overlooked by traditional financial institutions â and our plan is to serve millions more, and have been named by theÂ FortuneÂ Impact 20 list,Â CNBCâs Disruptor 50, andÂ ForbesâÂ Fintech 50 list for five years running. We are expanding across product offerings, countries and crypto and are looking for people who have an entrepreneurial spirit and are passionate about the mission.By creating a unique platform that enables lending and other financial services around the globe, people in emerging markets are able to start and expand small businesses, manage day-to-day needs, and pursue their financial goals with 